### munging json data

In [25]:
#put all packages here
import os
import glob
import pandas
import numpy
LOCAL_DATA_PATH = 'C:\Users\JoAnna\political_history\data'

In [31]:
#import all json files, concatenate into pandas dataframe
all_files = glob.glob(LOCAL_DATA_PATH + '/*.json')

df = pandas.concat((pandas.read_json(f, orient='index') for f in all_files))

print len(df)
df.head(50)

8069


,date,speaker,text,title
0,2012-10-03,National Economy,This debate and the next three—two Presidentia...,"Barack Obama: Presidential Debate in Denver, C..."
1,2012-10-03,National Economy,Thousands of people offered suggestions on seg...,"Barack Obama: Presidential Debate in Denver, C..."
10,2012-10-03,Laughter,But we all know that we've still got a lot of ...,"Barack Obama: Presidential Debate in Denver, C..."
100,2012-10-03,Gov. Romney.,You bet.,"Barack Obama: Presidential Debate in Denver, C..."
101,2012-10-03,Deficit and National Debt/Spending Cuts,None,"Barack Obama: Presidential Debate in Denver, C..."
102,2012-10-03,Mr. Lehrer.,And the question is this: What are the differe...,"Barack Obama: Presidential Debate in Denver, C..."
103,2012-10-03,Gov. Romney.,"Well, good. I'm glad you raised that, and it'...","Barack Obama: Presidential Debate in Denver, C..."
104,2012-10-03,Gov. Romney.,"So how do we deal with it? Well, mathematicall...","Barack Obama: Presidential Debate in Denver, C..."
105,2012-10-03,Gov. Romney.,The President would prefer raising taxes. I un...,"Barack Obama: Presidential Debate in Denver, C..."
106,2012-10-03,Gov. Romney.,"What things would I cut from spending? Well, f...","Barack Obama: Presidential Debate in Denver, C..."


In [32]:
#find unique speakers
print df['speaker'].head()
print df['speaker'].unique()

0                             National Economy 
1                             National Economy 
10                                     Laughter
100                                Gov. Romney.
101    Deficit and National Debt/Spending Cuts 
Name: speaker, dtype: object


TypeError: unhashable type: 'list'